# Can we classify open clusters and globular clusters using their B-V index and true diameter?

## Abstract

## Introduction

## Reading the Data

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [23]:
def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


In [25]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


# GLOBULAR CLUSTER DATA READING
glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 156, delim_whitespace = True, header = None)
glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 156, delim_whitespace = True, header = None)
glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 156, delim_whitespace = True, header = None)

# Combining the first two columns because the names were split
glob_data1["Cname"] = glob_data1.iloc[:,0] + glob_data1.iloc[:,1].astype(str)


glob_data_other = pd.concat([glob_data1["Cname"], glob_data1.iloc[:,4:10], glob_data1.iloc[:,12], glob_data2.iloc[:,4], glob_data3.iloc[:,8]], axis = 1)
glob_data_other.columns = ["Cname", "RA1", "RA2","RA3", "Dec1", "Dec2", "Dec3", "R_sun", "E(B-V)","r_h" ]


# OPEN CLUSTER DATA READING
open_data = pd.read_csv("open_data.tsv", skiprows = 39, delimiter = "\t", header = None)

# Code generated by ChatGPT-4o (OpenAI, 2024) - November 2024 version, Prompt: "I have too many whitespace missing values in my dataset, how do I remove them?"
open_data = open_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Remove leading/trailing spaces
open_data.replace('', pd.NA, inplace=True)  # Replace empty strings with NaN
open_data = open_data.dropna()

open_data.columns = ["RA", "Dec", "Diam", "Dist", "Age (log years)", "Metallicity", "E(B-V)", "Cname", "_RA.icrs", "_DE.icrs"]

print("Globular Data","\n")
print(glob_data_other.head())

print("Open Data", "\n")
print(open_data)

Globular Data 

      Cname    RA1   RA2    RA3  Dec1    Dec2   Dec3  R_sun  E(B-V)    r_h
0    NGC104   0.00  24.0   5.67 -72.0    4.00  52.60    4.5    0.04   0.36
1    NGC288  45.24 -26.0  34.00  57.4  152.30 -89.38   -0.1    0.03   1.35
2    NGC362  14.26 -70.0  50.00  55.6  301.53 -46.25    3.1    0.05   0.18
3  Whiting1  57.00  -3.0  15.00  10.0  161.22 -60.76  -13.9    0.03   7.93
4   NGC1261  16.21 -55.0  12.00  58.4  270.54 -52.12    0.1    0.01  17.73
Open Data 

          RA     Dec  Diam  Dist Age (log years) Metallicity E(B-V)  \
4    00 01.7  -30 12  89.0   190            7.70        0.03   0.09   
26   00 39.4  +85 04  13.0  1550            9.70       -0.06   0.04   
42   01 26.1  +63 03   4.4   900            9.10       -0.76   0.90   
60   01 54.8  +37 26  49.0   400            9.04       -0.21   0.02   
86   02 38.8  +42 34  35.0   440            8.29       -0.26   0.04   
..       ...     ...   ...   ...             ...         ...    ...   
909  20 32.5  +28 08  31.

/var/tmp/ipykernel_265005/1897360798.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 156, delim_whitespace = True, header = None)
/var/tmp/ipykernel_265005/1897360798.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 156, delim_whitespace = True, header = None)
/var/tmp/ipykernel_265005/1897360798.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 156, delim_whitespace = True, header = None)
/var/tmp/ipykernel_265005/1897360798.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use D

In [37]:
rows_part1 = []
rows_part2 = []

for i in range(0,68):
    rows_part1.append(i)

for i in range(228,591):
    rows_part1.append(i)
    
for i in range(0,248):
    rows_part2.append(i)

for i in range(408,591):
    rows_part2.append(i)

globular_part1 = pd.read_fwf("glob_data.txt",
                      skiprows=rows_part1)

globular_part2 = pd.read_fwf("glob_data.txt",
                      skiprows=rows_part2)

glob_data_full = pd.concat([globular_part1, globular_part2], axis=1)
glob_data = pd.concat([glob_data_full['B-V'],glob_data_full['R_Sun'], glob_data_other['r_h']], axis =  1)
glob_data

,B-V,R_Sun,r_h
0,0.88,4.5,0.36
1,0.65,8.9,1.35
2,0.77,8.6,0.18
3,NaN,30.1,7.93
4,0.72,16.3,17.73
...,...,...,...
152,0.66,11.5,0.32
153,0.60,8.1,0.06
154,1.07,19.0,19.21
155,0.76,26.0,0.48


In [36]:
glob_data['true diam'] = get_true_diameter(get_angular_size(glob_data['r_h']), glob_data['R_Sun'])

TypeError: can't multiply sequence by non-int of type 'float'